# Data Carpentry with `R`


Much of a data scientist's career is spent on data carpentry (also known as *data cleaning* or *tidying*. Pick your flavor) because datasets often come to us in a less than desirable format. Fortunately, there are several packages in `R` that make cleaning up our data a lot easier by providing functions to correct common messy dataset issues.

Before we begin, there are a couple of things that we need to understand about our "*tidy*" dataset. Tidy datasets take on the following 3 properties drawn from Hadley Wickham's [*Tidy Data*](../resources/tidy-data.pdf) paper:

1. *Variables are columns*
2. *Observations are rows*
3. *Values are cells*

Keeping these in mind, we can derive potential characteristics of messy data, such as values as column headers, variables as values, or multiple values per cell. 

This notebook is going to cover the backbone of data carpentry with `R` using `R`'s `tidyr` and `dplyr` packages.

---
### What we'll cover...
- [Spreading](#spread)
- [Gathering](#gather)
- [Separating](#separate)
- [Uniting](#unite)
---

## The Data

We are going to use some dummy data for demonstration purposes today and the examples of "messy" data are by no means exhaustive, but the following issues that we cover today are some very common problems when we get our hands on data.  

We'll begin by importing the libraries that we need as well as the data.

In [1]:
library(tidyr)
library(dplyr)
library(ggplot2)

weight <- read.csv("/dsa/data/all_datasets/dummy/pet_wt.csv")
chars <-read.csv("/dsa/data/all_datasets/dummy/pet_descr.csv")
breed <- read.csv("/dsa/data/all_datasets/dummy/pet_attr.csv")


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



<a id='spread'></a>
## Spreading

Take a look at the data frame below. Occasionally, when working with data, you will have a single observation that is distributed among several rows. In the table below, each dog can be considered an observation, but if you take a look, you can see that each individual dog takes up multiple rows. That is because **`char`** column contains variables as values and the **`description`** column contains the respective values. But remember, we want those variables to be columns...

In [3]:
chars

names,sex,char,description
<fct>,<fct>,<fct>,<fct>
Hermey,male,fur color,multi
Hermey,male,eye color,brown
Chewey,female,fur color,white
Chewey,female,eye color,brown
Pippa,female,fur color,white
Pippa,female,eye color,brown
Prairie,female,fur color,multi
Prairie,female,eye color,brown


`tidyr` provides a function for such a scenario. The `spread()` function takes three arguments, the first being the name of the data frame. The second and third parameters are described below:
- `key` : takes the name of the column that takes on the different variable types.
- `value`: takes the name of the column that contains the variable values.

For the `chars` data frame, the `char` column is the *`key`* and the `description` is the `value`.

In [4]:
spread(chars, key = `char`, value = `description`)

names,sex,eye color,fur color
<fct>,<fct>,<fct>,<fct>
Chewey,female,brown,white
Hermey,male,brown,multi
Pippa,female,brown,white
Prairie,female,brown,multi


<a id='gather'></a>
## Gathering

If `spread()` solves issues in which variables are values in a table, `gather()` solves the issues in which values are columns. In the table below, the columns **`zero`** and **`one`** represent ages at which weights were collected for each dog. In this case, "zero" and "one" are both values of a variable that we could call `age`, and the values under these columns could be considered values of the variable `weight`.

In [5]:
weight

names,zero,one
<fct>,<dbl>,<int>
Hermey,4.0,17
Chewey,5.0,22
Pippa,4.5,28
Prairie,3.0,19


The first thing to note when using the `gather()` function is that it takes on the sytax of `dplyr`'s `select()`. We are going to use the pipe function (`%>%`) in order to "gather" our values under a column. Taking a look at the example below, `weight` is the data frame that needs the carpentry and the `gather()` function takes three "types" of arguments. 
- The first arguments are the columns that are actually values (in this case, both **`zero`** and **`one`**)
- `key`: the name of the variable under which the new values need to be *gather*ed.
- `value`: the name of the variable that needs to be spread

In [6]:
weight %>% 
  gather(zero, one, key = "age", value = "weight")

names,age,weight
<fct>,<chr>,<dbl>
Hermey,zero,4.0
Chewey,zero,5.0
Pippa,zero,4.5
Prairie,zero,3.0
Hermey,one,17.0
Chewey,one,22.0
Pippa,one,28.0
Prairie,one,19.0


<a id='separate'></a>
## Separating

In a clean dataset, cells should only contain **1** value. Cells with more than one value will make data manipulations particularly difficult. Therefore, when it comes up, it is a good idea to separate the values. In the example below, there are two values under the column `attr` in each row.

In [7]:
breed

names,attr,month,day,year
<fct>,<fct>,<int>,<int>,<int>
Hermey,small/dachshund,12,24,1995
Chewey,small/jack russell terrier,10,11,2003
Pippa,medium/mix,1,14,2012
Prairie,small/mix,6,1,2013


To pull these values apart, we will use `tidyr`'s `separate()` function. `separate()` has three arguments that you will pass to it. The first is the column that you will be working with, in this case, the **`attr`** column. The other two parameters are as follows:
- `into`: takes a list of the column names that the values will be sorted into
- `sep`: is the seperator character

**`attr`** can be separated into "size" and "breed" and the values are deliniated by the "`/`" character.

In [8]:
breed %>%
separate(attr, into = c("size", "breed"), sep = "/")

names,size,breed,month,day,year
<fct>,<chr>,<chr>,<int>,<int>,<int>
Hermey,small,dachshund,12,24,1995
Chewey,small,jack russell terrier,10,11,2003
Pippa,medium,mix,1,14,2012
Prairie,small,mix,6,1,2013


<a id='unite'></a>
## Uniting

And of course, the opposite of separating is uniting. Occasionally you have a value that is spread across multiple columns. One such example is a date that is broken up into separate components. In most cases, it would make more sense if these components were contained under a single variable.

Take a look back at the `breed` data frame again...

In [9]:
breed

names,attr,month,day,year
<fct>,<fct>,<int>,<int>,<int>
Hermey,small/dachshund,12,24,1995
Chewey,small/jack russell terrier,10,11,2003
Pippa,medium/mix,1,14,2012
Prairie,small/mix,6,1,2013


...the **`month`**, **`day`** and **`year`** columns could all be under the umbrella variable of **`date`**. The `unite()` function does just that. It takes two types of arguments. The first are the columns that need to be *unite*d and the second is the `sep`erator that will go in between them. In the example below, we use the "-" to separate each date component.

In [11]:
breed %>% 
unite(birthdate,year,month,day, sep="-")

names,attr,birthdate
<fct>,<fct>,<chr>
Hermey,small/dachshund,1995-12-24
Chewey,small/jack russell terrier,2003-10-11
Pippa,medium/mix,2012-1-14
Prairie,small/mix,2013-6-1


In [10]:
head(breed)

names,attr,month,day,year
<fct>,<fct>,<int>,<int>,<int>
Hermey,small/dachshund,12,24,1995
Chewey,small/jack russell terrier,10,11,2003
Pippa,medium/mix,1,14,2012
Prairie,small/mix,6,1,2013


# Save your notebook, then `File > Close and Halt`